In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.5 MB/s eta 0:00:00


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon CPU @ 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 21.1/107.7 GB disk)


In [ ]:
from ultralytics import YOLO
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
!git clone https://github.com/matthias-k/DeepGaze


Cloning into 'DeepGaze'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 142 (delta 7), reused 2 (delta 2), pack-reused 132 (from 1)
Receiving objects: 100% (142/142), 892.66 KiB | 10.38 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
!wget -O /content/centerbias_mit1003.npy \
  https://github.com/matthias-k/DeepGaze/releases/download/v1.0.0/centerbias_mit1003.npy


--2025-12-14 09:27:42--  https://github.com/matthias-k/DeepGaze/releases/download/v1.0.0/centerbias_mit1003.npy
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/372933216/3c267f80-c32e-11eb-9f03-c6381f7da54a?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-14T10%3A03%3A50Z&rscd=attachment%3B+filename%3Dcenterbias_mit1003.npy&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-14T09%3A03%3A00Z&ske=2025-12-14T10%3A03%3A50Z&sks=b&skv=2018-11-09&sig=CfddLpr7sYFruZeSiwqh7ldIQJwwQCOOjGBokSKk8dY%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTcwNDc2MiwibmJmIjoxNzY1NzA0NDYyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJv

In [ ]:
import sys, cv2, torch, numpy as np
from scipy.ndimage import zoom
from scipy.special import logsumexp

if "/content/DeepGaze/deepgaze_pytorch" in sys.path:
    sys.path.remove("/content/DeepGaze/deepgaze_pytorch")
if "/content/DeepGaze" not in sys.path:
    sys.path.append("/content/DeepGaze")      # ou deepgaze_pytorch selon ton clone
import deepgaze_pytorch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Charger le modèle
model_s = deepgaze_pytorch.DeepGazeIIE(pretrained=True).to(DEVICE).eval()

# 2) Charger ton image (depuis OpenCV par ex.)
image_bgr = cv2.imread("/content/test_bicycle.png")
def create_saillance(image_bgr):
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  H, W = image_rgb.shape[:2]

  # 3) Charger le centerbias pré-computé et l’adapter à la taille de l’image
  centerbias_template = np.load("/content/centerbias_mit1003.npy")   # 1024x1024
  centerbias = zoom(
      centerbias_template,
      (H / centerbias_template.shape[0], W / centerbias_template.shape[1]),
      order=0,
      mode="nearest",
  )
  # normaliser en log-densité
  centerbias -= logsumexp(centerbias)

  # 4) Convertir en tenseurs
  image_tensor = torch.from_numpy(image_rgb.copy()).permute(2, 0, 1).float().unsqueeze(0).to(DEVICE)
  # FIX: Removed one unsqueeze(0). The DeepGaze Finalizer expects centerbias as (Batch, Height, Width).
  centerbias_tensor = torch.from_numpy(centerbias).unsqueeze(0).float().to(DEVICE)

  # 5) Prédire la log-densité de saillance
  with torch.no_grad():
      log_density = model_s(image_tensor, centerbias_tensor)  # <-- ici on passe bien centerbias
      saliency = log_density.exp()[0, 0].cpu().numpy()      # carte de saillance (H x W)
  sal_norm = (saliency - saliency.min()) / (saliency.max() - saliency.min())
  sal_norm = (sal_norm * 255).astype(np.uint8)

  # Convertir la heatmap en BGR avec colormap JET
  heatmap = cv2.applyColorMap(sal_norm, cv2.COLORMAP_JET)

  # Lire image originale au même format
  orig = image_bgr # Use the already loaded image_bgr
  orig = cv2.resize(orig, (saliency.shape[1], saliency.shape[0]))

  # Fusion
  overlay = cv2.addWeighted(orig, 0.6, heatmap, 0.4, 0)
  final_overlay = cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)
  return final_overlay


Downloading: "https://bitbucket.org/robert_geirhos/texture-vs-shape-pretrained-models/raw/60b770e128fffcbd8562a3ab3546c1a735432d03/resnet50_finetune_60_epochs_lr_decay_after_30_start_resnet50_train_45_epochs_combined_IN_SF-ca06340c.pth.tar" to /root/.cache/torch/hub/checkpoints/resnet50_finetune_60_epochs_lr_decay_after_30_start_resnet50_train_45_epochs_combined_IN_SF-ca06340c.pth.tar


100%|██████████| 195M/195M [00:11<00:00, 17.5MB/s]


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth


100%|██████████| 117M/117M [00:00<00:00, 214MB/s]


Loaded pretrained weights for efficientnet-b5
Downloading: "https://github.com/pytorch/vision/zipball/v0.6.0" to /root/.cache/torch/hub/v0.6.0.zip


...

100%|██████████| 77.4M/77.4M [00:00<00:00, 165MB/s]
Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


100%|██████████| 95.8M/95.8M [00:00<00:00, 141MB/s]


Downloading: "https://github.com/matthias-k/DeepGaze/releases/download/v1.0.0/deepgaze2e.pth" to /root/.cache/torch/hub/checkpoints/deepgaze2e.pth


100%|██████████| 400M/400M [00:05<00:00, 77.1MB/s]


In [ ]:
cap = cv2.VideoCapture("/content/eye_gaze_merged.mp4")
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # ou 'XVID', 'avc1', etc.
out = cv2.VideoWriter("/content/output.avi", fourcc, fps, (width, height))


start_frame = 2220
with open("/content/datagaze.txt", 'r') as f:
    data = f.readlines()
    f.close()
x = np.zeros(int(len(data)/2))
y = np.zeros(int(len(data)/2))
for i in range(int(len(data)/2)):
    data[i] = data[i][:-2]
    x[i]= float(data[i].split(";")[0])
    y[i]= float(data[i].split(";")[1])
print(len(x))
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

count_bicycle = 0
count = 0
count_total = 0

# Read the first frame outside the loop
ret, frame = cap.read()

while ret:
    image = np.asarray(frame).copy() # Make a writable copy of the image array
    results = model.predict(image)

    image = create_saillance(image)
    # Process detections in the current frame
    for box in results[0].boxes.data:
        # Check if the detected object is a bicycle (assuming class ID 1 for bicycle)
        if int(box[5]) == 1:
            # Extract bounding box coordinates from the current 'box'
            x_min, y_min, x_max, y_max = box[:4].int().tolist()

            # Check if gaze is within the bicycle's bounding box
            # Ensure 'count' (index for x, y gaze data) does not exceed bounds of x, y arrays
            if count < len(x) and x_min < x[count_total+start_frame] < x_max and y_min < y[count_total+start_frame] < y_max:
                count_bicycle += 1
            count += 1 # Increment count for each bicycle detected and processed for gaze

            # Define the two points for the rectangle in the correct format (x, y)
            pt1 = (x_min, y_min)
            pt2 = (x_max, y_max)

            cv2.rectangle(image, pt1, pt2, (255,0,0), 2)  # Draw rectangle

    out.write(image)
    count_total += 1
    print(f"frame : {count_total}")

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Read the next frame at the end of the loop
    ret, frame = cap.read()

# Release resources outside the loop
out.release()
cap.release()
print("Video processing finished.")

# Avoid division by zero if no bicycles were processed
if count > 0:
    print(f"vélo regardé pendant : {count_bicycle / count}")
else:
    print("No bicycles were detected or processed for gaze.")

2893

0: 384x640 2 bicycles, 2 cars, 324.4ms
Speed: 15.9ms preprocess, 324.4ms inference, 43.0ms postprocess per image at shape (1, 3, 384, 640)
frame : 1

0: 384x640 2 bicycles, 2 cars, 147.3ms
Speed: 8.3ms preprocess, 147.3ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
frame : 2
Video processing finished.
vélo regardé pendant : 0.5


In [ ]:
from google.colab import files
files.download("/content/output.avi")
